In [157]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import string
import unicodedata
from pathlib import Path
import collections
import itertools
import math

from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading Data

In [3]:
# name data from: https://download.pytorch.org/tutorial/data.zip

In [4]:
if not Path("names").is_dir():
  !curl "https://download.pytorch.org/tutorial/data.zip" -o "data.zip"
  !unzip -j "data.zip" "data/names/*" -d "names"

In [5]:
all_chars = string.ascii_letters + " .,;'-"
print(all_chars)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'-


In [6]:
def sanitise_line(line):
    # https://stackoverflow.com/a/518232/5013267
    return ''.join(
        c for c in unicodedata.normalize('NFD', line)
        if unicodedata.category(c) != 'Mn'
        and c in all_chars
    )

In [7]:
name_data = {}
for path in Path("names").rglob("*.txt"):
    with open(path, "r", encoding="utf8") as f:
        names = []
        for line in f.read().strip().split("\n"):
            names.append(sanitise_line(line))
        name_data[path.stem] = names
    
    print(f"Loaded: {path}")

Loaded: names/Arabic.txt
Loaded: names/Chinese.txt
Loaded: names/Czech.txt
Loaded: names/Dutch.txt
Loaded: names/English.txt
Loaded: names/French.txt
Loaded: names/German.txt
Loaded: names/Greek.txt
Loaded: names/Irish.txt
Loaded: names/Italian.txt
Loaded: names/Japanese.txt
Loaded: names/Korean.txt
Loaded: names/Polish.txt
Loaded: names/Portuguese.txt
Loaded: names/Russian.txt
Loaded: names/Scottish.txt
Loaded: names/Spanish.txt
Loaded: names/Vietnamese.txt


# Data Preparation Stuff

In [8]:
class OneHotTranslator:
    def __init__(self, elements):
        self.elements = elements
        self.n_elements = len(self.elements)
        
    def index_to_vec(self, index):
        return F.one_hot(torch.tensor([index]), num_classes=self.n_elements)[0]
    
    def index_from_vec(self, vec):
        return vec.argmax()
    
    def elm_to_vec(self, elm):
        return self.index_to_vec(self.elements.index(elm))
    
    def elm_from_vec(self, vec):
        return self.elements[int(self.index_from_vec(vec))]
    
    def __len__(self):
        return self.n_elements
    
    def __getitem__(self, val):
        if isinstance(val, int):
            return self.index_to_vec(val)
        else:
            return self.elm_to_vec(val)

In [9]:
category_translator = OneHotTranslator(tuple(name_data.keys()))
print(category_translator.elements)
print("len:", len(category_translator))

('Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese')
len: 18


In [10]:
char_translator = OneHotTranslator(["<SOS>", "<EOS>"] + list(all_chars))
print(char_translator.elements)
print("len:", len(char_translator))

['<SOS>', '<EOS>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ', '.', ',', ';', "'", '-']
len: 60


In [111]:
# convenience functions
def build_name(name, fixed_length):
    """
    Build a list of one-hot encoded vectors representing the given name string
    """
    ret = [char_translator.elm_to_vec(char) for char in (["<SOS>"] + list(name) + ["<EOS>"])]
    
    while len(ret) < fixed_length:
        ret.append(char_translator.elm_to_vec("<EOS>"))
    
    return ret

def unbuild_name(t_chars):
    """
    Inverse of build_name
    """
    return "".join([char_translator.elm_from_vec(vec) for vec in t_chars])

def build_category(category):
    """
    Build a one-hot encoded vector representing the given category string
    """
    return category_translator.elm_to_vec(category)

def unbuild_category(vec):
    """
    Inverse of build_category
    """
    return category_translator.elm_from_vec(vec)

In [112]:
# torch.stack(build_name("hi", 10)).shape

In [119]:
# simple Dataset class wrapper around the above variables
class NamesDataset(Dataset):
    def __init__(self, max_length):
        self.max_length = max_length
        self._data_pairs = [(cat, name) for cat in name_data for name in name_data[cat]]
    
    def __len__(self):
        return len(self._data_pairs)
    
    def __getitem__(self, index):
        cat, name = self._data_pairs[index]
        return build_category(cat), build_name(name, self.max_length)

# Neural

In [120]:
class SeqModel(nn.Module):
    def __init__(self, prime_size, input_size, hidden_size):
        super().__init__()
        self.prime_size = prime_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTMCell(self.input_size + self.prime_size, self.hidden_size)
        
        # NB: output size = input size
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        self.dropout = nn.Dropout(0.1)
        self.lsoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self, t_prime, t_input, t_hidden, t_cell):
        # t_prime:     batches of vectors for the LSTM to be primed on
        #              shape: (N, prime_size)
        #
        # t_input:     shape: (N, input_size)
        #              t_input[0]: one-hot vector encoding of a character
        #
        # t_hidden:    shape: (N, hidden_size)
        #
        # t_cell:      shape: (N, hidden_size)
        
        # lstm_input:  t_prime and t_input concated together
        #              shape: (N, prime_size + input_size)
        lstm_input = torch.cat((t_prime, t_input), dim=1)
        t_hidden, t_cell = self.lstm(lstm_input.float(), (t_hidden, t_cell))
        
        # t_output:    shape: (N, input_size)
        #              NB: t_output's shape = t_input's shape
        t_output = self.fc(self.dropout(t_hidden))
        t_output = self.lsoftmax(t_output)
        
        return t_output, t_hidden, t_cell
    
    def init_hidden(self, batch_size=1):
        t_hidden = torch.zeros(batch_size, self.hidden_size)
        t_cell = torch.zeros(batch_size, self.hidden_size)
        
        return t_hidden, t_cell

In [121]:
model = SeqModel(len(category_translator), len(char_translator), 32)
model.to(device)

SeqModel(
  (lstm): LSTMCell(78, 32)
  (fc): Linear(in_features=32, out_features=60, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (lsoftmax): LogSoftmax(dim=1)
)

In [274]:
# test_category_index = 0

# t_prime = category_translator[test_category_index].view(1, len(category_translator))
# print("t_prime shape:\t", t_prime.shape)

# t_input = char_translator["<SOS>"].view(1, len(char_translator))
# print("t_input shape:\t", t_input.shape)

# t_hidden, t_cell = model.init_hidden(1)
# print("t_hidden shape:\t", t_hidden.shape)
# print("t_cell shape:\t", t_cell.shape)

In [275]:
# t_output, t_hidden, t_cell = model.forward(t_prime, t_input, t_hidden, t_cell)
# print("t_output shape:\t", t_output.shape)
# print("t_hidden shape:\t", t_hidden.shape)
# print("t_cell shape:\t", t_cell.shape)

# Eval

In [280]:
def generate(max_length, trial_category_name="German", N=3):
    with torch.no_grad():
        t_prime = category_translator[trial_category_name].unsqueeze(0).repeat((N, 1)).to(device)
        t_char = char_translator["<SOS>"].unsqueeze(0).repeat((N, 1)).to(device)
        t_hidden, t_cell = map(lambda x: x.to(device), model.init_hidden(N))

        output_dict = collections.defaultdict(str)
        for char_index in range(max_length):
            t_output, t_hidden, t_cell = model(t_prime, t_char, t_hidden, t_cell)
            
            for n in range(N):
                output_dict[n] += char_translator.elm_from_vec(t_output[n])
        
        return [output_dict[n] for n in range(N)]

# Training

In [277]:
max_length = max([len(name) for name in itertools.chain(*name_data.values())]) + 2  # +2 for SOS and EOS
print(max_length)

22


In [278]:
sample_count = len(tuple(itertools.chain(*name_data.values())))
print(sample_count)

20074


In [279]:
# config
num_epochs = 100
num_batches = math.ceil(sample_count/batch_size) * num_epochs
learning_rate = 0.0003
plot_limit = None
batch_size = 64

In [189]:
train_dataloader = DataLoader(NamesDataset(max_length), batch_size=batch_size, shuffle=True, collate_fn=lambda x: x)

In [190]:
g = iter(train_dataloader)

In [191]:
batch = next(g)

In [192]:
t_prime = torch.stack([sample[0] for sample in batch])
t_prime.shape

torch.Size([64, 18])

In [193]:
t_hidden, t_cell = model.init_hidden(64)
print(t_hidden.shape)

torch.Size([64, 32])


In [204]:
for char_index in range(max_length):
    print("char_index: ", char_index)
    t_char = torch.stack([sample[1][char_index] for sample in batch])
    print(t_char.shape)

char_index:  0
torch.Size([64, 60])
char_index:  1
torch.Size([64, 60])
char_index:  2
torch.Size([64, 60])
char_index:  3
torch.Size([64, 60])
char_index:  4
torch.Size([64, 60])
char_index:  5
torch.Size([64, 60])
char_index:  6
torch.Size([64, 60])
char_index:  7
torch.Size([64, 60])
char_index:  8
torch.Size([64, 60])
char_index:  9
torch.Size([64, 60])
char_index:  10
torch.Size([64, 60])
char_index:  11
torch.Size([64, 60])
char_index:  12
torch.Size([64, 60])
char_index:  13
torch.Size([64, 60])
char_index:  14
torch.Size([64, 60])
char_index:  15
torch.Size([64, 60])
char_index:  16
torch.Size([64, 60])
char_index:  17
torch.Size([64, 60])
char_index:  18
torch.Size([64, 60])
char_index:  19
torch.Size([64, 60])
char_index:  20
torch.Size([64, 60])
char_index:  21
torch.Size([64, 60])


In [205]:
losses = collections.deque(maxlen=plot_limit)

In [206]:
criterion = nn.NLLLoss()
optimiser = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.01)
# optimiser = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [207]:
t_char.shape

torch.Size([64, 60])

In [208]:
t_output, t_hidden, t_cell = model(t_prime, t_char, t_hidden, t_cell)

In [209]:
t_output.shape

torch.Size([64, 60])

In [210]:
batch = next(g)

In [286]:
torch.stack([sample[0] for sample in batch]).to(device).size(0)

42

In [289]:
for batch_no, batch in enumerate(train_dataloader):
    t_prime = torch.stack([sample[0] for sample in batch]).to(device)
    t_hidden, t_cell = map(lambda x: x.to(device), model.init_hidden(t_prime.size(0)))
    
    model.zero_grad()
    loss = 0
    for char_index in range(max_length - 1):
        t_char = torch.stack([sample[1][char_index] for sample in batch]).to(device)
        t_output, t_hidden, t_cell = model(t_prime, t_char, t_hidden, t_cell)
        
        t_target = torch.stack([sample[1][char_index + 1].argmax() for sample in batch]).to(device)
        loss += criterion(t_output, t_target)
    
    loss.backward()
    optimiser.step()
    
    name_loss = loss.item() / (len(name) - 1)
    losses.append(name_loss)
    
    if batch_no + 1 >= num_batches:
        break
    plt.figure()
    plt.plot(losses, scalex=True)
    plt.title("Loss")
    plt.grid(True)
    plt.show()
    
    print("batch_no: {} epoch: {} loss: {:.06f}\n{}".format(
        batch_no, 
        batch_no*batch_size//sample_count, 
        name_loss,
        "\n".join(generate(max_length))
    ))
    clear_output(wait=True)
        

KeyboardInterrupt: 

# Post-Training Eval

In [291]:
generate(max_length)

['aaaaaa<EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS>',
 'aaaaaa<EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS>',
 'aaaaaa<EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS>']